## Introduction
This notebook is an example of a RAG using an InMemory volatile vector database. That will allow us to see the whole process of setting up a RAG before deploying it to Azure in a more industrialized way.

We will use the [Kernel Memory (KM)](https://microsoft.github.io/kernel-memory/) which is a multi-modal AI Service specialized in the efficient indexing of documents and information through custom continuous data pipelines, with support for Retrieval Augmented Generation (RAG), synthetic memory, prompt engineering, and custom semantic memory processing.

KM supports PDF and Word documents, PowerPoint presentations, Images, Spreadsheets and more, extracting information and generating memories by leveraging Large Language Models (LLMs), Embeddings and Vector storage.

### Login to Azure
First, you need to login to your Azure account. You can do this by running the following command and following the instructions that will be displayed:
```bash
az login
```

It's needed as we will be using Managed Identity to authenticate to Azure services.

In [17]:
az login > null
Write-Host("Successfully logged in to Azure!");

Successfully logged in to Azure!


### Install the required nuget packages

In [18]:
#r "nuget: Microsoft.KernelMemory.Core"
#r "nuget: dotenv.net"

Installed Packages dotenv.net, 3.1.3 Microsoft.KernelMemory.Core, 0.35.240321.1

### Add the neccecary using statements

In [19]:
using Microsoft.KernelMemory;
using dotenv.net;
using System;
using System.IO;

### Load the environment variables from the .env file
- Rename the file .env.sample to .env
- Add the required values to the .env file

In [20]:
DotEnv.Load();
var env = DotEnv.Read();

### Setup of the Kernel Memory using InMemory Serverless Vector Database

In [21]:
var endpoint = env["AZURE_OPENAI_ENDPOINT"];
var config = new AzureOpenAIConfig(){
    APIType = AzureOpenAIConfig.APITypes.ChatCompletion,
    Auth = AzureOpenAIConfig.AuthTypes.AzureIdentity,
    Endpoint = endpoint,
    Deployment = env["AZURE_OPENAI_CHAT_DEPLOYMENT"],
};
var embdeddingConfig = new AzureOpenAIConfig(){
    APIType = AzureOpenAIConfig.APITypes.EmbeddingGeneration,
    Auth = AzureOpenAIConfig.AuthTypes.AzureIdentity,
    Endpoint = endpoint,
    Deployment = env["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"],
};

var memory = new KernelMemoryBuilder()
    .WithAzureOpenAITextGeneration(config)
    .WithAzureOpenAITextEmbeddingGeneration(embdeddingConfig)
    .Build<MemoryServerless>();

### Read the documents from data folder and import them in the memory vector database

In [22]:
string folderPath = "../data";
string[] pdfFiles = Directory.GetFiles(folderPath, "*.pdf", SearchOption.AllDirectories);

foreach (string filePath in pdfFiles)
{
    string fileName = Path.GetFileName(filePath);
    string fullPath = Path.GetFullPath(filePath);
    
    await memory.ImportDocumentAsync(fullPath, documentId: fileName);

    Console.WriteLine("Successfully imported File Name: " + fileName);
}

Successfully imported File Name: Benefit_Options.pdf
Successfully imported File Name: employee_handbook.pdf
Successfully imported File Name: role_library.pdf


### Utility function that ask question to the Memory Vector Database and return the answer provided by Azure OpenAI model

In [23]:
async Task AskQuestionAsync(string question)
{
    var answer = await memory.AskAsync(question);
    Console.WriteLine($"Question: {question}\n\nAnswer: {answer.Result}");

    Console.WriteLine("\nSources:");

    foreach (var x in answer.RelevantSources)
    {
        Console.WriteLine($"  - {x.SourceName}  - {x.Link} [{x.Partitions.First().LastUpdate:D}]");
    }
}

### Some questions and answers

In [24]:
await AskQuestionAsync("What's in your knowledge base?");
await AskQuestionAsync("What are the benefits options for employees?");
await AskQuestionAsync("Fourni moi 5 questions à poser sur les avantages sociaux des employés.");

Question: What's in your knowledge base?

Answer: In my knowledge base, I have information about Contoso Electronics, including details about their employee handbook, company values, performance reviews, and various job roles such as Director of Research and Development, Vice President of Human Resources, Vice President of Research and Development, Vice President of Product Management, Manager of Human Resources, Manager of Research and Development, Manager of Product Management, Director of Sales, Director of Marketing, Manager of Marketing, and Manager of Operations. Additionally, I have information about data security, privacy policies, whistleblower policies, and workplace violence prevention and response.

Sources:
  - employee_handbook.pdf  - default/employee_handbook.pdf/5cdbb1aa86084ce1b3054b87832d4451 [Friday, April 5, 2024]
  - role_library.pdf  - default/role_library.pdf/d8f22330d164409b8b00e1972f8fe7d4 [Friday, April 5, 2024]
Question: What are the benefits options for empl

### A question that cannot be answered as the response is in an image 
- The response is integrated in the image and there is no OCR implemented yet
- It will be implemented in the future
![Employee's cost per paycheck](../docs/images/00_cost_per_employee.png)


In [25]:
await AskQuestionAsync("What is the employee's cost per paycheck?");

Question: What is the employee's cost per paycheck?

Answer: INFO NOT FOUND

Sources:
  - employee_handbook.pdf  - default/employee_handbook.pdf/5cdbb1aa86084ce1b3054b87832d4451 [Friday, April 5, 2024]
  - Benefit_Options.pdf  - default/Benefit_Options.pdf/08fd84ff01624f4d84e9a6715a1b25e0 [Friday, April 5, 2024]
  - role_library.pdf  - default/role_library.pdf/d8f22330d164409b8b00e1972f8fe7d4 [Friday, April 5, 2024]
